In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "hotdogFlavoredWater!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

def convert_age_to_weeks(age_str):
    if pd.isna(age_str):
        return None
    if 'year' in age_str:
        return int(age_str.split()[0]) * 52  # Approximate: 1 year = 52 weeks
    elif 'months' in age_str:
        return int(age_str.split()[0]) * 4  # Approximate: 1 month = 4 weeks
    elif 'weeks' in age_str:
        return int(age_str.split()[0])  # Already in weeks
    return None  # Handle unexpected formats

# Convert the age strings to numeric weeks
df['age_upon_outcome'] = df['age_upon_outcome'].apply(convert_age_to_weeks)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Drop Down Layout
app.layout = html.Div([
    html.H2('Richard Wellman', id="unique_id"),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),

    # Dropdown for filtering
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
            {'label': 'Disaster/Individual Tracking', 'value': 'Disaster/Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        clearable=False
    ),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),
    
    # Create a container for the pie chart and map side by side
    html.Div([
        html.Div(id='graph-id', style={'flex': '1', 'padding': '10px'}),  # Left side for pie chart
        html.Div(id='map-id', style={'flex': '1', 'padding': '10px'})  # Right side for map
    ], style={'display': 'flex'})  # Flexbox layout for side-by-side display
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Check if the filter type is valid
    if filter_type is None or filter_type == 'Reset':
        filtered_df = df  # Return the full DataFrame if no filter is selected
    else:
        # Filter based on dropdown selection
        if filter_type == 'Water Rescue':
            filtered_df = df[
                (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                (df['sex_upon_outcome'] == 'Intact Female') &
                (df['age_upon_outcome'].between(26, 156))
            ]
        elif filter_type == 'Mountain/Wilderness Rescue':
            filtered_df = df[
                (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome'].between(26, 156)) 
            ]
        elif filter_type == 'Disaster/Individual Tracking':
            filtered_df = df[
                (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome'].between(20, 300))
            ]
    
    # Return the filtered data in the required format for the DataTable
    return filtered_df.to_dict('records')  # Converts DataFrame to a list of dictionaries

# Pie Chart Method
@app.callback(
    Output('graph-id', 'children'),
    [Input('filter-type', 'value'),
     Input('datatable-id', 'data')]  # Listen to the DataTable's data
)
def update_graphs(filter_type, viewData):

    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if the DataFrame is empty
    if dff.empty:
        return [html.P("No data available for the selected filter.")]  # Informative message if no data

    # Create a pie chart of the preferred animals
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    
    return [dcc.Graph(figure = fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "derived_virtual_selected_rows"),
     Input('filter-type', 'value')]
)

def update_map(viewData, selected_rows, filter_type):

    # Create a DataFrame from the viewData
    dff = pd.DataFrame(viewData)

    # Handle row selection after reset or normal operations
    if selected_rows is not None and len(selected_rows) > 0:
        # Get the selected row index
        selected_row_index = selected_rows[0]
        selected_row = dff.iloc[selected_row_index]

        # Get the latitude and longitude for the selected animal
        lat = selected_row['location_lat']
        lon = selected_row['location_long']

        # Create a single marker for the selected animal
        marker = dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(selected_row['name']),
            dl.Popup([html.H1(selected_row['name']),
                      html.P(f"Type: {selected_row['animal_type']}, Breed: {selected_row['breed']}, Color: {selected_row['color']}")])
            ])        

        # Center the map on the selected animal's location
        center = [lat, lon]

        # Create the map with the selected animal's marker
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=center, zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                marker  # Only the selected animal's marker
            ])]        
    
    # Default return for no selection
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.2672, -97.7431], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
    ])]


    
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10352/
